## Exploración de variables individuales

Se exploraran de manera individual las variables que estan relacionadas a la cartera de activos de un cliente para ver como es que estos estan relacionados con la adquisicion de un prestamo a plazo.

Esto con el objetivo de despues crear una nueva variable mediante **Ingenieria de caracteristicas** para conseguir una nueva caracteristica que pueda describir mejor estos bienes.

In [1]:
import pandas as pd

In [2]:
file_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter03/bank-full.csv'

In [3]:
dataBank = pd.read_csv(file_url, sep=';')

In [4]:
dataBank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
dataBank.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [6]:
#Relacion entre 'housing' y que tan propenso es que se adquiera un prestamo

dataBank.groupby(['housing', 'y'])['y'].agg(houseTot='count').reset_index()


,housing,y,houseTot
0,no,no,16727
1,no,yes,3354
2,yes,no,23195
3,yes,yes,1935


In [7]:
#Persona que no cuentan con una casa y compran un prestamo
print('Personas sin casa que compraron el prestamo: '+str((3354*100.0)/(16727+3354)) + '%')
#Personas que tienen casa y comprar una casa
print('Personas con casa que compraron el prestamo: '+str((1935*100.0)/(23195+1935))+ '%')

Personas sin casa que compraron el prestamo: 16.70235546038544%
Personas con casa que compraron el prestamo: 7.699960206923995%


En cuanto a proporciones, podemos ver que las personas que no tienen casa son mas propensas a comprar estos prestamos que los que ya tienen casa.

In [8]:
# Relacion entre las personas que tienen un prestamos contra la posible compra
dataBank.groupby(['loan', 'y'])['y'].agg(loanTot='count').reset_index()

,loan,y,loanTot
0,no,no,33162
1,no,yes,4805
2,yes,no,6760
3,yes,yes,484


In [9]:
#Persona que no cuentan con prestamo y compran un deposito a plazo fijo
print('Personas sin prestamos que compraron el deposito: '+str((4805*100.0)/(33162+4805)) + '%')
#Personas que tienen un prestamo y compran un deposito a plazo fijo
print('Personas con prestamos que compraron el deposito: '+str((484*100.0)/(6760+484))+ '%')

Personas sin prestamos que compraron el deposito: 12.655727342165564%
Personas con prestamos que compraron el deposito: 6.681391496410822%


Como podemos ver tanto si tiene una casa o no, o si cuenta con un prestamo o no, son variables caetgoricas, las cuales son faciles de encontrar relaciones, esto cambia cuando la variable es numerica.

Una estrategia comun puede ser convertir estos datos continuos en datos ordinarios y buscar atraves de las categorias derivadas.

In [10]:
#Para convertir estos datos continuos, primero debemos encontrar los quantiles
#Estos se pueden encontrar de manera directa con el metodo describe(), pero otra forma es
#con numpy
import numpy as np
q25, q50, q75 = np.quantile(dataBank['balance'], [0.25, 0.5, 0.75])
print(q25,q50,q75)

72.0 448.0 1428.0


In [11]:
#Lo que nos dan como informacion los quantiles es como estan distribuidos los datos, por debajo de que valor
#se encuentran.
dataBank['balanceClass'] = 'Quant1'
dataBank.loc[(dataBank['balance'] > q25) & (dataBank['balance'] <= q50), 'balanceClass'] = 'Quant2'
dataBank.loc[(dataBank['balance'] > q50) & (dataBank['balance'] <= q75), 'balanceClass'] = 'Quant3'
dataBank.loc[dataBank['balance'] > q75, 'balanceClass'] = 'Quant4'
dataBank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,balanceClass
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,Quant4
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,Quant1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,Quant1
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,Quant4
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,Quant1


In [12]:
# Calculating los clientes en cada uno de los quantiles
balanceTot = dataBank.groupby(['balanceClass'])['y'].agg(balanceTot='count').reset_index()
balanceTot

,balanceClass,balanceTot
0,Quant1,11317
1,Quant2,11291
2,Quant3,11306
3,Quant4,11297


In [13]:
# Calculamos de los clientes por cada cuantil quienes son los que si compraron y los que no
balanceProp = dataBank.groupby(['balanceClass', 'y'])['y'].agg(balanceCat='count').reset_index()
balanceProp

,balanceClass,y,balanceCat
0,Quant1,no,10498
1,Quant1,yes,819
2,Quant2,no,10061
3,Quant2,yes,1230
4,Quant3,no,9891
5,Quant3,yes,1415
6,Quant4,no,9472
7,Quant4,yes,1825


In [14]:
# Haciendo un join de ambos
balanceComb = pd.merge(balanceProp, balanceTot, on = ['balanceClass'])
balanceComb['catProp'] = (balanceComb.balanceCat / balanceComb.balanceTot)*100
balanceComb

,balanceClass,y,balanceCat,balanceTot,catProp
0,Quant1,no,10498,11317,92.763100
1,Quant1,yes,819,11317,7.236900
2,Quant2,no,10061,11291,89.106368
3,Quant2,yes,1230,11291,10.893632
4,Quant3,no,9891,11306,87.484521
5,Quant3,yes,1415,11306,12.515479
6,Quant4,no,9472,11297,83.845269
7,Quant4,yes,1825,11297,16.154731


Con esto podemos dar cuenta que conforme los clientes tienen un mayor balance estos son mas propensos a comprar estos depositos a plazos fijos